In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.6287708917339145, 0.6707862165765671)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_six_month'].values

## columns 
cols = KY_X.columns

In [7]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 0.08, cols, 816)
single_stump_model['features']

['age_at_current_charge19.0',
 'age_at_current_charge22.0',
 'age_at_current_charge25.0',
 'age_at_current_charge26.0',
 'age_at_current_charge36.0',
 'age_at_current_charge48.0',
 'age_at_current_charge62.0',
 'p_arrest5',
 'p_charges3',
 'p_violence3',
 'p_felony3',
 'p_misdemeanor3',
 'p_property2',
 'p_property3',
 'p_weapon1',
 'p_felprop_viol1',
 'p_dui1',
 'p_fta_two_year1',
 'p_incarceration1',
 'six_month1',
 'one_year1',
 'three_year1',
 'current_violence1']

In [8]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary['best_params'], stumps_summary['KY_score'], stumps_summary['auc_diffs']

({'C': 0.02}, 0.6052558111644094, 0.00021833729683562453)

### RiskSLIM

In [10]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_six_month'].values

## columns 
cols = KY_X.columns

In [22]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 
                                        0.06, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

15

In [23]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [24]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'misdemeanor_six_month', 
                                max_coef = 5, 
                                max_coef_number =5, 
                                max_runtime=2000,
                                max_offset = 100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 10:14 PM | 203 rows in lookup table
11/12/19 @ 10:14 PM | ------------------------------------------------------------
11/12/19 @ 10:14 PM | runnning initialization procedure
11/12/19 @ 10:14 PM | ------------------------------------------------------------
11/12/19 @ 10:14 PM | CPA produced 2 cuts
11/12/19 @ 10:14 PM | running naive rounding on 4 solutions
11/12/19 @ 10:14 PM | best objective value: 0.6931
11/12/19 @ 10:14 PM | rounding produced 4 integer solutions
11/12/19 @ 10:14 PM | best objective value is 0.6931
11/12/19 @ 10:14 PM | running sequential rounding on 4 solutions
11/12/19 @ 10:14 PM | best objective value: 0.6931
11/12/19 @ 10:14 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 10:14 PM | polishing 4 solutions
11/12/19 @ 10:14 PM | best objective value: 0.6931
11/12/19 @ 10:14 PM | polishing produced 4 integer solutions
11/12/19 @ 10:14 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:14 PM | adding 254 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3790.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 132154 20303        cutoff              0.3737        0.3691   817519    1.23%           rho_2 U 132154  10041     21
 134892 20068        0.3708    14        0.3737        0.3693   829930    1.18%           rho_0 U 134892 110952     27
 137619 19800        0.3701    10        0.3737        0.3694   842052    1.14%          rho_12 U 137619  25330     29
 140297 19422        0.3723     9        0.3737        0.3696   854323    1.09%          rho_12 D 140297 140296     30
 143048 18985        0.3733     9        0.3737        0.3698   866343    1.04%           rho_2 D 143048 143047     27
 145707 18393        cutoff              0.3737        0.3700   878194    0.99%        alpha_14 U 145707 145706     32
 148412 17845        cutoff              0.3737        0.3702   889969    0.95%           rho_4 U 148412  76101     26
 151257 17180        0.3720    13        0.3737        0.3704   902417    0.89%           rho_0 U 151257 127776     20
Elapsed time = 39.34 sec. (49618.86 ticks, tree 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:15 PM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3688.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

  98080  6304        0.3589     7        0.3591        0.3566   595749    0.69%           rho_3 U  98080  90901     23
 100269  5339        cutoff              0.3591        0.3570   604095    0.58%          rho_12 U 100269  57634     30
Elapsed time = 33.63 sec. (40412.31 ticks, tree = 1.97 MB, solutions = 10)
 102438  4249        cutoff              0.3591        0.3575   612072    0.46%           rho_7 D 102438 102436     33
 104596  3007        cutoff              0.3591        0.3579   619249    0.33%          rho_12 D 104596  45837     20
 106721  1529        cutoff              0.3591        0.3585   625949    0.18%           rho_0 D 106721  97403     20

User cuts applied:  707

Root node processing (before b&c):
  Real time             =    0.06 sec. (1.77 ticks)
Sequential b&c:
  Real time             =   35.55 sec. (44067.31 ticks)
                          ------------
Total (root+branch&cut) =   35.61 sec. (44069.08 ticks)
+----------------------------------------------+--

...

 151080  6503        cutoff              0.3810        0.3789   952260    0.56%          rho_14 U 151080  53111     22
 154423  4344        cutoff              0.3810        0.3794   965506    0.41%           rho_7 U 154423 136480     23
 157704  1823        cutoff              0.3810        0.3802   977002    0.21%           rho_5 D 157704 122454     26

User cuts applied:  418

Root node processing (before b&c):
  Real time             =    0.06 sec. (1.77 ticks)
Sequential b&c:
  Real time             =   36.25 sec. (44688.05 ticks)
                          ------------
Total (root+branch&cut) =   36.31 sec. (44689.82 ticks)
+----------------------------------------------+-------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-2 + score))   |                   |           |
| ============================================ | ================= | ========= |
| age_at_current_charge25.0                    |          1 points |   + ..... |
| p_misdemeanor3                       

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:17 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3725.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 105919 21962        0.3631    10        0.3649        0.3567   700757    2.25%           rho_0 U 105919 105918     41
 108105 22019        0.3616    14        0.3649        0.3569   712471    2.20%           rho_6 U 108105 108104     21
Elapsed time = 32.47 sec. (41261.94 ticks, tree = 6.72 MB, solutions = 9)
 110349 22087        0.3574    14        0.3649        0.3570   724363    2.16%           rho_6 D 110349  70997     28
 112528 22161        0.3594     9        0.3649        0.3572   735893    2.11%         alpha_2 D 112528 112527     51
 114686 22153        cutoff              0.3649        0.3573   747308    2.07%           rho_9 U 114686   4766     19
 116889 22198        0.3647    13        0.3649        0.3575   758593    2.03%           rho_4 D 116889 116888     32
 119071 22175        cutoff              0.3649        0.3577   769879    1.98%           rho_4 U 119071 119070     26
 121228 22101        cutoff              0.3649        0.3578   780985    1.93%           rho

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:18 PM | adding 260 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3777.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 101897 10924        0.3685    19        0.3709        0.3673   667509    0.98%           rho_7 D 101897  61338     24
 103965 10211        cutoff              0.3709        0.3676   678194    0.91%          rho_15 U 103965  93623     17
 106064  9430        cutoff              0.3709        0.3679   688151    0.83%           rho_6 U 106064  56902     23
 108161  8562        cutoff              0.3709        0.3682   698015    0.75%          rho_11 D 108161  81968     34
 110238  7570        cutoff              0.3709        0.3685   707421    0.66%           rho_4 D 110238 100465     20
 112285  6507        0.3704    12        0.3709        0.3688   716659    0.57%           rho_4 D 112285  50995     37
 114300  5278        cutoff              0.3709        0.3692   725215    0.46%           rho_8 D 114300   8597     28
 116288  3861        cutoff              0.3709        0.3696   733307    0.35%           rho_8 U 116288   8896     30
 118282  2303        cutoff              0.3709 

In [25]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_validation_score'])

(0.5209933099949828, 0.5541531574314542)

In [26]:
#### save results
summary_misdemeanor6_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}

In [27]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Six Month\\"
results = [["Misdemeanor",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Six Month FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)